In [1]:
from src.tokenizers.services.available_tokenizers import available_tokenizers
from src.common.models.dataloader_type import DataloaderType
import torch
from src.nn.services.decoding_strat import GreedyDAGStrategy

g:\Projects\Visual Studio Code\LM\.venv\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
info = available_tokenizers['helsinki_en_zh']()

In [3]:
from src.nn.architectures.dag_encoder_decoder import EncoderDecoderDAG

In [4]:
checkpoint = torch.load("checkpoints/wmt19_en_zh_dag_dropout_0.12/model.pt")

In [5]:
tokenizer = info.tokenizer

In [6]:
state_dict = checkpoint['model_state_dict']
kwargs = checkpoint['model_kwargs']

In [7]:
device = "cpu"

In [8]:
model = EncoderDecoderDAG(
    **kwargs
)

In [9]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
model = model.to(device)

In [11]:
model.eval()

EncoderDecoderDAG(
  (embedding): StableEmbedding(
    (embedding): Embedding(65003, 384)
  )
  (pos_embedding_enc): StableEmbedding(
    (embedding): Embedding(52, 384)
  )
  (pos_embedding_dec): StableEmbedding(
    (embedding): Embedding(208, 384)
  )
  (enc_layers): ModuleList(
    (0-2): 3 x TransformerBlock(
      (attn): TransformerAttention(
        (attn_proj): Linear(in_features=384, out_features=1152, bias=True)
        (out_proj): Linear(in_features=384, out_features=384, bias=True)
        (attn_dropout): Dropout(p=0.12, inplace=False)
        (out_dropout): Dropout(p=0.12, inplace=False)
      )
      (ffn): Linear(in_features=384, out_features=1152, bias=True)
      (ffn2): Linear(in_features=1152, out_features=384, bias=True)
      (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ffn_activation): GELU(approximate='none')
      (ffn_dropout): Dropout(p=0.12, inplace=False)
      (ff

In [32]:
sample = input("Enter a sample: ")
encoded = tokenizer.encode(sample)
bos_idx = info.bos_idx
eos_idx = info.eos_idx
encoded = [bos_idx] + encoded + [eos_idx]
encoded = torch.tensor(encoded).unsqueeze(0)
strat = GreedyDAGStrategy(info, device)
coeff = 4
inferenced = model.naive_inference(encoder_input=encoded, strat=strat, coefficient=coeff)
print(tokenizer.decode(inferenced[0].tolist()))

<unk> 福州是港口城市</s> </s>
